<table>
    <tr>
        <td><img src="./img/Macc.png" width="auto"/></td>
        <td>
            <table><tr>
            <h1 style="color:blue;text-align:center">Lógica para Ciencias de la Computación</h1></td>
            </tr></table>   
        <td>&nbsp;</td>
        <td>
            <table><tr>
            <tp><p style="font-size:150%;text-align:center">Grupo 6</p></tp>
            <tp><p style="font-size:150%;text-align:center">Representación del problema</p></tp>
            </tr></table>
        </td>
    </tr>
</table>

---

# Objetivo <a class="anchor" id="inicio"></a>

Implementar en Python la representación del problema que nuestro grupo desea abordar, esto de acuerdo a lo establecido en la primera entrega y a las modificaciones de la misma

# Secciones

1. [Problema a resolver.](#prob)
2. [Descriptores.](#des)
3. [Implementación de restricciones.](#imp)

# Problema a resolver <a class="anchor" id="prob"></a>

([Volver al inicio](#inicio))

Trabajaremos sobre el siguiente problema. Se busca ubicar cierto número de $X$ en un tablero de 3 en linea de tamaño 7x7 de manera que no queden tres $X$ seguidas en ninguna dirección.

Un ejemplo de una posible solución al problema sería la siguiente

![ejemplo](img/tabla.png)

Este problema cuenta con tres restricciones en total:

1. Solo se permite una $X$ por casilla.
2. No deben haber tres $X$ seguidas en ninguna dirección (horizontal, vertical o diagonales).
3. Deben haber mínimo tres $X$ en cada fila y columna.


# Descriptores <a class="anchor" id="des"></a>

([Volver al inicio](#inicio))

Para empezar, deben definirse las letras proposicionales, las cuales definen la posición de cada $X$ en el tablero:

$XenC_{x,y}$ es verdadera sii hay una $X$ ubicada en la casilla $(x,y)$, siendo $x$ las filas y $y$ las columnas

Para hacer la implementación en python, se usa la clase `Descriptor`, que se encuentra en la librería `Logica`.  Esta clase permite codificar un átomo, representándolo como un solo caracter (es decir, una cadena de longitud 1).

Importamos la clase a partir de la librería:

In [18]:
from Logica import *

Ahora creamos un descriptor de dos argumentos, siendo ambos para representar la casilla $(x,y)$:

In [19]:
Nx = 7
Ny = 7
X = list(range(Nx))
Y = list(range(Ny))
XenC = Descriptor([Nx, Ny])

### Codificando en un solo caracter

Mediante el método P() podemos crear las codificaciones. A continuación presentaremos cada uno de los caracteres que codifican los cruces de información de que una $X$ se encuentre en una casilla $(x,y)$ (donde $x,y\in\{0, 1, 2, 3, 4, 5, 6\}$):

In [20]:
print("Cantidad de átomos XenC:", XenC.rango[1] - XenC.rango[0])
print("Caracteres correspondientes a los átomos XenC:\n")
for x in range(Nx):
    for y in range(Ny):
        atomo = XenC.P([x,y])
        print(f"Hay una X en la casilla ({X[x]},{Y[y]}) es el átomo {atomo}")
    print("")

Cantidad de átomos XenC: 49
Caracteres correspondientes a los átomos XenC:

Hay una X en la casilla (0,0) es el átomo Ā
Hay una X en la casilla (0,1) es el átomo ć
Hay una X en la casilla (0,2) es el átomo Ď
Hay una X en la casilla (0,3) es el átomo ĕ
Hay una X en la casilla (0,4) es el átomo Ĝ
Hay una X en la casilla (0,5) es el átomo ģ
Hay una X en la casilla (0,6) es el átomo Ī

Hay una X en la casilla (1,0) es el átomo ā
Hay una X en la casilla (1,1) es el átomo Ĉ
Hay una X en la casilla (1,2) es el átomo ď
Hay una X en la casilla (1,3) es el átomo Ė
Hay una X en la casilla (1,4) es el átomo ĝ
Hay una X en la casilla (1,5) es el átomo Ĥ
Hay una X en la casilla (1,6) es el átomo ī

Hay una X en la casilla (2,0) es el átomo Ă
Hay una X en la casilla (2,1) es el átomo ĉ
Hay una X en la casilla (2,2) es el átomo Đ
Hay una X en la casilla (2,3) es el átomo ė
Hay una X en la casilla (2,4) es el átomo Ğ
Hay una X en la casilla (2,5) es el átomo ĥ
Hay una X en la casilla (2,6) es el átomo 

### Decodificando el caracter

Ahora podemos incluir un método para visualizar más fácilmente la información que porta cada letra proposicional. Esto es, en nuestro ejemplo en cuestión, nuestro objeto `XenC` representa el cruce de información de que una $X$ encuentra en un lugar. Entonces, al decodificar una letra proposicional, queremos que nos presente claramente esta información y no un caracter inpronunciable (aunque es precisamente este caracter el que usará la máquina). 

Para ello podemos usar el siguiente método `escribir`: 

In [21]:
def escribir(self, literal):
    if '-' in literal:
        atomo = literal[1:]
        neg = 'No h'
    else:
        atomo = literal
        neg = 'H'
    x, y= self.inv(atomo)
    return f"{neg}ay una X en la casilla ({X[x]},{Y[y]})"
    
from types import MethodType

XenC.escribir = MethodType(escribir, XenC)

---

# Implementación de restricciones <a class="anchor" id="imp"></a>

([Volver al inicio](#inicio))

Ahora es necesario crear las reglas que limitarán los valores de verdad para las letras proposicionales. En nuestro problema tenemos tres restricciones:

1. Solo se permite una $X$ por casilla.
2. No deben haber tres $X$ seguidas en ninguna dirección (horizontal, vertical o diagonales).
3. Deben haber mínimo tres $X$ en cada fila y columna.

---

## Restricción 1

Comencemos por considerar la restricción 1, que dice que sólo se permite una $X$ casilla. Note que esta restricción ya se cumple con la implementación del descriptor:

$$\large XenC_{x,y}$$

Ya que el mismo se asegura de que en cierta posición solo se encuentre una sola $X$

---

## Restricción 2

Ahora consideremos la restricción 2, que dice no puede haber 3 $X$ seguidas en ninguna dirección.

En este caso, se deben revisar 4 direcciones, siendo horizontal, vertical y ambas diagonales. Para comenzar,  se define cada dirección por separado:

---

### Horizontal ($\rightarrow$)

La fórmula para representar que no pueden haber 3 $X$ seguidas en horizontal es la siguiente:

$$\large \bigwedge_{x \in Filas\atop { y \in\{0, 4\}}}\biggl(XenC_{x, y}\wedge XenC_{x, y+1}\to\neg\left(XenC_{x, y+2}\right)\biggr)$$

Esta fórmula se implementa en Python de la siguiente manera:

In [22]:
x = 0
y = 0
Direcciones = []
formula1 = ''
inicial = True
for x in X:
    for y in range(5):
        if inicial:
            formula1 = '(' + '(' + XenC.P([x,y]) + 'Y' + XenC.P([x,y+1]) + ')' + '>-' + XenC.P([x,y+2]) + ')'
            inicial = False
        else:
            formula1 = "(" + formula1 + "Y" + '(' + '(' + XenC.P([x,y]) + 'Y' + XenC.P([x,y+1]) + ')' + '>-' + XenC.P([x,y+2]) + '))'

print(formula1)
print(visualizar_formula(formula1, XenC))
Direcciones.append(formula1)
A = inorder_to_tree(formula1)

((((((((((((((((((((((((((((((((((((ĀYć)>-Ď)Y((ćYĎ)>-ĕ))Y((ĎYĕ)>-Ĝ))Y((ĕYĜ)>-ģ))Y((ĜYģ)>-Ī))Y((āYĈ)>-ď))Y((ĈYď)>-Ė))Y((ďYĖ)>-ĝ))Y((ĖYĝ)>-Ĥ))Y((ĝYĤ)>-ī))Y((ĂYĉ)>-Đ))Y((ĉYĐ)>-ė))Y((ĐYė)>-Ğ))Y((ėYĞ)>-ĥ))Y((ĞYĥ)>-Ĭ))Y((ăYĊ)>-đ))Y((ĊYđ)>-Ę))Y((đYĘ)>-ğ))Y((ĘYğ)>-Ħ))Y((ğYĦ)>-ĭ))Y((ĄYċ)>-Ē))Y((ċYĒ)>-ę))Y((ĒYę)>-Ġ))Y((ęYĠ)>-ħ))Y((ĠYħ)>-Į))Y((ąYČ)>-ē))Y((ČYē)>-Ě))Y((ēYĚ)>-ġ))Y((ĚYġ)>-Ĩ))Y((ġYĨ)>-į))Y((ĆYč)>-Ĕ))Y((čYĔ)>-ě))Y((ĔYě)>-Ģ))Y((ěYĢ)>-ĩ))Y((ĢYĩ)>-İ))
((((((((((((((((((((((((((((((((((((Hay una X en la casilla (0,0) Y Hay una X en la casilla (0,1)) >  no Hay una X en la casilla (0,2)) Y ((Hay una X en la casilla (0,1) Y Hay una X en la casilla (0,2)) >  no Hay una X en la casilla (0,3))) Y ((Hay una X en la casilla (0,2) Y Hay una X en la casilla (0,3)) >  no Hay una X en la casilla (0,4))) Y ((Hay una X en la casilla (0,3) Y Hay una X en la casilla (0,4)) >  no Hay una X en la casilla (0,5))) Y ((Hay una X en la casilla (0,4) Y Hay una X en la casilla (0,5)) >  no Hay una X en la casilla

---

### Vertical ($\downarrow$)

La fórmula para representar que no pueden haber 3 $X$ seguidas en vertical es la siguiente:

$$\large \bigwedge_{x \in\{0, 4\}\atop { y \in Columnas}}\biggl(XenC_{x, y}\wedge XenC_{x+1, y}\to\neg\left(XenC_{x+2, y}\right)\biggr)$$

Esta fórmula se implementa en Python de la siguiente manera:


In [24]:
x = 0
y = 0
formula2 = ''
inicial = True
for y in Y:
    for x in range(5):
        if inicial:
            formula2 = '(' + '(' + XenC.P([x,y]) + 'Y' + XenC.P([x+1,y]) + ')' + '>-' + XenC.P([x+2,y]) + ')'
            inicial = False
        else:
            formula2 = "(" + formula2 + "Y" + '(' + '(' + XenC.P([x,y]) + 'Y' + XenC.P([x+1,y]) + ')' + '>-' + XenC.P([x+2,y]) + '))'

print(formula2)
print(visualizar_formula(formula2, XenC))
Direcciones.append(formula2)

((((((((((((((((((((((((((((((((((((ĀYā)>-Ă)Y((āYĂ)>-ă))Y((ĂYă)>-Ą))Y((ăYĄ)>-ą))Y((ĄYą)>-Ć))Y((ćYĈ)>-ĉ))Y((ĈYĉ)>-Ċ))Y((ĉYĊ)>-ċ))Y((ĊYċ)>-Č))Y((ċYČ)>-č))Y((ĎYď)>-Đ))Y((ďYĐ)>-đ))Y((ĐYđ)>-Ē))Y((đYĒ)>-ē))Y((ĒYē)>-Ĕ))Y((ĕYĖ)>-ė))Y((ĖYė)>-Ę))Y((ėYĘ)>-ę))Y((ĘYę)>-Ě))Y((ęYĚ)>-ě))Y((ĜYĝ)>-Ğ))Y((ĝYĞ)>-ğ))Y((ĞYğ)>-Ġ))Y((ğYĠ)>-ġ))Y((ĠYġ)>-Ģ))Y((ģYĤ)>-ĥ))Y((ĤYĥ)>-Ħ))Y((ĥYĦ)>-ħ))Y((ĦYħ)>-Ĩ))Y((ħYĨ)>-ĩ))Y((ĪYī)>-Ĭ))Y((īYĬ)>-ĭ))Y((ĬYĭ)>-Į))Y((ĭYĮ)>-į))Y((ĮYį)>-İ))
((((((((((((((((((((((((((((((((((((Hay una X en la casilla (0,0) Y Hay una X en la casilla (1,0)) >  no Hay una X en la casilla (2,0)) Y ((Hay una X en la casilla (1,0) Y Hay una X en la casilla (2,0)) >  no Hay una X en la casilla (3,0))) Y ((Hay una X en la casilla (2,0) Y Hay una X en la casilla (3,0)) >  no Hay una X en la casilla (4,0))) Y ((Hay una X en la casilla (3,0) Y Hay una X en la casilla (4,0)) >  no Hay una X en la casilla (5,0))) Y ((Hay una X en la casilla (4,0) Y Hay una X en la casilla (5,0)) >  no Hay una X en la casilla

---

### Diagonal Derecha ($\searrow$)

La fórmula para representar que no pueden haber 3 $X$ seguidas en diagonal derecha ($\searrow$) es la siguiente:

$$\large \bigwedge_{x \in\{0, 4\}\atop { y \in\{0, 4\}}}\biggl(XenC_{x, y}\wedge XenC_{x+1, y+1}\to\neg\left(XenC_{x+2, y+2}\right)\biggr)$$

Esta fórmula se implementa en Python de la siguiente manera:


In [25]:
x = 0
y = 0
formula3 = ''
inicial = True
for x in range(5):
    for y in range(5):
        if inicial:
            formula3 = '(' + '(' + XenC.P([x,y]) + 'Y' + XenC.P([x+1,y+1]) + ')' + '>-' + XenC.P([x+2,y+2]) + ')'
            inicial = False
        else:
            formula3 = "(" + formula3 + "Y" + '(' + '(' + XenC.P([x,y]) + 'Y' + XenC.P([x+1,y+1]) + ')' + '>-' + XenC.P([x+2,y+2]) + '))'

print(formula3)
print(visualizar_formula(formula3, XenC))
Direcciones.append(formula3)

((((((((((((((((((((((((((ĀYĈ)>-Đ)Y((ćYď)>-ė))Y((ĎYĖ)>-Ğ))Y((ĕYĝ)>-ĥ))Y((ĜYĤ)>-Ĭ))Y((āYĉ)>-đ))Y((ĈYĐ)>-Ę))Y((ďYė)>-ğ))Y((ĖYĞ)>-Ħ))Y((ĝYĥ)>-ĭ))Y((ĂYĊ)>-Ē))Y((ĉYđ)>-ę))Y((ĐYĘ)>-Ġ))Y((ėYğ)>-ħ))Y((ĞYĦ)>-Į))Y((ăYċ)>-ē))Y((ĊYĒ)>-Ě))Y((đYę)>-ġ))Y((ĘYĠ)>-Ĩ))Y((ğYħ)>-į))Y((ĄYČ)>-Ĕ))Y((ċYē)>-ě))Y((ĒYĚ)>-Ģ))Y((ęYġ)>-ĩ))Y((ĠYĨ)>-İ))
((((((((((((((((((((((((((Hay una X en la casilla (0,0) Y Hay una X en la casilla (1,1)) >  no Hay una X en la casilla (2,2)) Y ((Hay una X en la casilla (0,1) Y Hay una X en la casilla (1,2)) >  no Hay una X en la casilla (2,3))) Y ((Hay una X en la casilla (0,2) Y Hay una X en la casilla (1,3)) >  no Hay una X en la casilla (2,4))) Y ((Hay una X en la casilla (0,3) Y Hay una X en la casilla (1,4)) >  no Hay una X en la casilla (2,5))) Y ((Hay una X en la casilla (0,4) Y Hay una X en la casilla (1,5)) >  no Hay una X en la casilla (2,6))) Y ((Hay una X en la casilla (1,0) Y Hay una X en la casilla (2,1)) >  no Hay una X en la casilla (3,2))) Y ((Hay una X en la casill

---

### Diagonal Izquierda ($\swarrow$)

La fórmula para representar que no pueden haber 3 $X$ seguidas en diagonal izquierda ($\swarrow$) es la siguiente:

$$\large \bigwedge_{x \in\{0, 4\}\atop { y \in\{2, 6\}}}\biggl(XenC_{x, y}\wedge XenC_{x+1, y-1}\to\neg\left(XenC_{x+2, y-2}\right)\biggr)$$

Esta fórmula se implementa en Python de la siguiente manera:


In [26]:
x = 0
y = 0
formula4 = ''
inicial = True
for x in range(5):
    for y in range(2,7):
        if inicial:
            formula4 = '(' + '(' + XenC.P([x,y]) + 'Y' + XenC.P([x+1,y-1]) + ')' + '>-' + XenC.P([x+2,y-2]) + ')'
            inicial = False
        else:
            formula4 = "(" + formula4 + "Y" + '(' + '(' + XenC.P([x,y]) + 'Y' + XenC.P([x+1,y-1]) + ')' + '>-' + XenC.P([x+2,y-2]) + '))'

print(formula4)
print(visualizar_formula(formula4, XenC))
Direcciones.append(formula4)

((((((((((((((((((((((((((ĎYĈ)>-Ă)Y((ĕYď)>-ĉ))Y((ĜYĖ)>-Đ))Y((ģYĝ)>-ė))Y((ĪYĤ)>-Ğ))Y((ďYĉ)>-ă))Y((ĖYĐ)>-Ċ))Y((ĝYė)>-đ))Y((ĤYĞ)>-Ę))Y((īYĥ)>-ğ))Y((ĐYĊ)>-Ą))Y((ėYđ)>-ċ))Y((ĞYĘ)>-Ē))Y((ĥYğ)>-ę))Y((ĬYĦ)>-Ġ))Y((đYċ)>-ą))Y((ĘYĒ)>-Č))Y((ğYę)>-ē))Y((ĦYĠ)>-Ě))Y((ĭYħ)>-ġ))Y((ĒYČ)>-Ć))Y((ęYē)>-č))Y((ĠYĚ)>-Ĕ))Y((ħYġ)>-ě))Y((ĮYĨ)>-Ģ))
((((((((((((((((((((((((((Hay una X en la casilla (0,2) Y Hay una X en la casilla (1,1)) >  no Hay una X en la casilla (2,0)) Y ((Hay una X en la casilla (0,3) Y Hay una X en la casilla (1,2)) >  no Hay una X en la casilla (2,1))) Y ((Hay una X en la casilla (0,4) Y Hay una X en la casilla (1,3)) >  no Hay una X en la casilla (2,2))) Y ((Hay una X en la casilla (0,5) Y Hay una X en la casilla (1,4)) >  no Hay una X en la casilla (2,3))) Y ((Hay una X en la casilla (0,6) Y Hay una X en la casilla (1,5)) >  no Hay una X en la casilla (2,4))) Y ((Hay una X en la casilla (1,2) Y Hay una X en la casilla (2,1)) >  no Hay una X en la casilla (3,0))) Y ((Hay una X en la casill

### Fórmula general

Para satisfacer completamente la restricción, se deben unir las 4 fórmulas anteriores de la siguiente manera:

$$\normalsize \bigwedge_{x \in Filas\atop { y \in\{0, 4\}}}\biggl(XenC_{x, y}\wedge XenC_{x, y+1}\to\neg\left(XenC_{x, y+2}\right)\biggr) \wedge \bigwedge_{x \in\{0, 4\}\atop { y \in Columnas}}\biggl(XenC_{x, y}\wedge XenC_{x+1, y}\to\neg\left(XenC_{x+2, y}\right)\biggr) \wedge \bigwedge_{x \in\{0, 4\}\atop { y \in\{0, 4\}}}\biggl(XenC_{x, y}\wedge XenC_{x+1, y+1}\to\neg\left(XenC_{x+2, y+2}\right)\biggr) \wedge \bigwedge_{x \in\{0, 4\}\atop { y \in\{2, 6\}}}\biggl(XenC_{x, y}\wedge XenC_{x+1, y-1}\to\neg\left(XenC_{x+2, y-2}\right)\biggr)$$

Esta fórmula se implementa en Python de la siguiente manera:

In [27]:
regla2 = Ytoria(Direcciones)
print(regla2)
print(visualizar_formula(regla2, XenC))

((((((((((((((((((((((((((((((((((((((((ĀYć)>-Ď)Y((ćYĎ)>-ĕ))Y((ĎYĕ)>-Ĝ))Y((ĕYĜ)>-ģ))Y((ĜYģ)>-Ī))Y((āYĈ)>-ď))Y((ĈYď)>-Ė))Y((ďYĖ)>-ĝ))Y((ĖYĝ)>-Ĥ))Y((ĝYĤ)>-ī))Y((ĂYĉ)>-Đ))Y((ĉYĐ)>-ė))Y((ĐYė)>-Ğ))Y((ėYĞ)>-ĥ))Y((ĞYĥ)>-Ĭ))Y((ăYĊ)>-đ))Y((ĊYđ)>-Ę))Y((đYĘ)>-ğ))Y((ĘYğ)>-Ħ))Y((ğYĦ)>-ĭ))Y((ĄYċ)>-Ē))Y((ċYĒ)>-ę))Y((ĒYę)>-Ġ))Y((ęYĠ)>-ħ))Y((ĠYħ)>-Į))Y((ąYČ)>-ē))Y((ČYē)>-Ě))Y((ēYĚ)>-ġ))Y((ĚYġ)>-Ĩ))Y((ġYĨ)>-į))Y((ĆYč)>-Ĕ))Y((čYĔ)>-ě))Y((ĔYě)>-Ģ))Y((ěYĢ)>-ĩ))Y((ĢYĩ)>-İ))Y((((((((((((((((((((((((((((((((((((ĀYā)>-(Ă))Y((āYĂ)>-(ă)))Y((ĂYă)>-(Ą)))Y((ăYĄ)>-(ą)))Y((ĄYą)>-(Ć)))Y((ćYĈ)>-(ĉ)))Y((ĈYĉ)>-(Ċ)))Y((ĉYĊ)>-(ċ)))Y((ĊYċ)>-(Č)))Y((ċYČ)>-(č)))Y((ĎYď)>-(Đ)))Y((ďYĐ)>-(đ)))Y((ĐYđ)>-(Ē)))Y((đYĒ)>-(ē)))Y((ĒYē)>-(Ĕ)))Y((ĕYĖ)>-(ė)))Y((ĖYė)>-(Ę)))Y((ėYĘ)>-(ę)))Y((ĘYę)>-(Ě)))Y((ęYĚ)>-(ě)))Y((ĜYĝ)>-(Ğ)))Y((ĝYĞ)>-(ğ)))Y((ĞYğ)>-(Ġ)))Y((ğYĠ)>-(ġ)))Y((ĠYġ)>-(Ģ)))Y((ģYĤ)>-(ĥ)))Y((ĤYĥ)>-(Ħ)))Y((ĥYĦ)>-(ħ)))Y((ĦYħ)>-(Ĩ)))Y((ħYĨ)>-(ĩ)))Y((ĪYī)>-(Ĭ)))Y((īYĬ)>-(ĭ)))Y((ĬYĭ)>-(Į)))Y((ĭYĮ)>-(į)))Y((ĮYį)>-(İ))))Y(((((((((((((((((((

---

## Restricción 3

La tercera restricción indica que deben haber como mínimo 3 $X$ en cada fila y columna del tablero. Para satisfacer esta restricción, se debe implementar la misma en horizontal y vertical.

$$\bigwedge_{x\in\{0,1\}}\bigwedge_{y\in\{0,1\}}\bigwedge_{n\in Numeros}\left(OenCasilla_{x,y,n}\to\neg\left(\bigvee_{m\neq n} OenCasilla_{x,y,m}\right)\right)$$


**Ejercicio 3:** Implemente la fórmula anterior como un string de Python.

In [ ]:
formula1 = ''
formula_n = '' 
formula_y = ''
formula_x = ''
for x in X:
    for y in Y:
        for n in numeros:
            numero = n
            otros_numeros = [m for m in numeros if m != n]
            inicial = True
            for numero in otros_numeros:
                m = numero
                if inicial:
                    formula1 = OenCasilla.P([x,y,m])
                    inicial = False
                else:
                    formula1 = "(" + formula1 + "O" + OenCasilla.P([x,y,m]) + ")"
            formula1 = "(" + OenCasilla.P([x,y,n]) + ">-" + formula1 + ")"
            if n==0:
                formula_n = formula1
            else:
                formula_n = "(" + formula_n + "Y" + formula1 + ")"
        if y==0:
            formula_y = formula_n
        else:
            formula_y = "(" + formula_y + "Y" + formula_n + ")"
    if x==0:
        formula_x = formula_y
    else:
        formula_x = "(" + formula_x + "Y" + formula_y + ")"

print(formula_x + "\n")
print(visualizar_formula(formula_x, OenCasilla))
                

---